# Reference

- [1] [waveforms-sdk-getting-started](https://digilent.com/reference/test-and-measurement/guides/waveforms-sdk-getting-started?srsltid=AfmBOor7O9UOChKs5_BxxQbqcvO2yG6K-rLJxbPG9qBuXNIQOY_XfWvY)

# Workflow

## 1. Importing the Constants and Loading the Dynamic Library

In [1]:
import ctypes                     # import the C compatible data types
from sys import platform, path    # this is needed to check the OS type and get the PATH
from os import sep                # OS specific file path separators
 
# load the dynamic library, get constants path (the path is OS specific)
if platform.startswith("win"):
    # on Windows
    dwf = ctypes.cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
    # constants_path = "C:/Program Files (x86)/Digilent/WaveFormsSDK/samples/py"
elif platform.startswith("darwin"):
    # on macOS
    lib_path = sep + "Library" + sep + "Frameworks" + sep + "dwf.framework" + sep + "dwf"
    dwf = ctypes.cdll.LoadLibrary(lib_path)
    constants_path = sep + "Applications" + sep + "WaveForms.app" + sep + "Contents" + sep + "Resources" + sep + "SDK" + sep + "samples" + sep + "py"
else:
    # on Linux
    dwf = ctypes.cdll.LoadLibrary("libdwf.so")
    constants_path = sep + "usr" + sep + "share" + sep + "digilent" + sep + "waveforms" + sep + "samples" + sep + "py"
 
# import constants
print(constants_path)
path.append(constants_path)
import dwfconstants as constants

C:\Program Files (x86)\Digilent\WaveFormsSDK\samples\py


In [2]:
print(constants.enumfilterAll)
print(dwf)

c_long(0)
<CDLL 'dwf', handle 7ffbfa300000 at 0x23c2a8dbf80>


## 2. Connecting the Test & Measurement Device

In [ ]:
class data:
    """
        stores the device handle and the device name
    """
    handle = ctypes.c_int(0)
    name = ""
def open(device=None, config=0):
    """
        open a specific device

        parameters: - device type: None (first device), "Analog Discovery", "Analog Discovery 2", "Analog Discovery Studio", "Digital Discovery", "Analog Discovery Pro 3X50", "Analog Discovery Pro 5250"
                    - configuration: 0 = auto, default = auto

        returns:    - device data
    """
    device_names = [("Analog Discovery", constants.devidDiscovery), ("Analog Discovery 2", constants.devidDiscovery2),
                    ("Analog Discovery Studio", constants.devidDiscovery2), ("Digital Discovery", constants.devidDDiscovery),
                    ("Analog Discovery Pro 3X50", constants.devidADP3X50), ("Analog Discovery Pro 5250", constants.devidADP5250)]
    
    # decode device names
    device_type = constants.enumfilterAll
    for pair in device_names:
        if pair[0] == device:
            device_type = pair[1]
            break

    # count devices
    device_count = ctypes.c_int()
    print(f"device cnt: {device_count}")
    dwf.FDwfEnum(device_type, ctypes.byref(device_count))
    print(f"after dwf.FDwfEnum, device cnt: {device_count}")

    # check for connected devices
    if device_count.value <= 0:
        if device_type.value == 0:
            raise error("There are no connected devices", "open", "device")
        else:
            raise error("Error: There is no " + device + " connected", "open", "device")

    # this is the device handle - it will be used by all functions to "address" the connected device
    device_handle = ctypes.c_int(0)

    # connect to the first available device
    index = 0
    while device_handle.value == 0 and index < device_count.value:
        dwf.FDwfDeviceConfigOpen(ctypes.c_int(index), ctypes.c_int(config), ctypes.byref(device_handle))
        index += 1  # increment the index and try again if the device is busy

    # check connected device type
    device_name = ""
    print(f"device handle value: {device_handle.value}")
    if device_handle.value != 0:
        device_id = ctypes.c_int()
        device_rev = ctypes.c_int()
        dwf.FDwfEnumDeviceType(ctypes.c_int(index - 1), ctypes.byref(device_id), ctypes.byref(device_rev))

        # decode device id
        for pair in device_names:
            if pair[1].value == device_id.value:
                device_name = pair[0]
                break

    # check for errors
    print(f"constant hdw: {constants.hdwfNone}")

    global data
    data.handle = device_handle
    data.name = device_name
    # data = __get_info__(data)
    return data

### Testing connection to the device

In [ ]:
# NOTE: "device handle value" MUST NOT BE 0
#       to show a successful connection to the Analog Device 3 (AD3)      
device_data = open()

device cnt: c_long(0)
after dwf.FDwfEnum, device cnt: c_long(1)
device handle value: 1
constant hdw: c_long(0)


## 3. Using Instruments

**All of the codes from section 1 and section 2 of Workflow** are contained the python package **WF_SDK**.

**[Link to **WF_SDK** package, go to this page](https://digilent.com/reference/test-and-measurement/guides/waveforms-sdk-getting-started?srsltid=AfmBOor7O9UOChKs5_BxxQbqcvO2yG6K-rLJxbPG9qBuXNIQOY_XfWvY)** - section **Creating Modules**, look for the text "..You can download the archive containing the module and some test files **here**..." to download WF_SDK

**Before running the code in this section**: if the codes from section 1 and 2 of Workflow have been run before, **MUST RESTART** this jupyternotebook, so that the connection to hardware AD3 created by section 1 and 2 of Workflow can be **CLOSED**. Otherwise, the code of this section 3 can not take handle of the hardware AD3.

- REFERENCE

  - [1] [import python libraries from any directory](https://www.geeksforgeeks.org/python-import-module-from-different-directory/)

  - [2] Ask GPT: how to import a module containing `__init__.py` from any directory into the current working directory of a python file.

In [ ]:
# importing sys
import sys

# adding path to **parent folder** of package **WF_SDK** the  to the system path
sys.path.insert(0, 'C:/Users/20245580/LabCode/AD3/WaveForms-SDK-Getting-Started-PY-master/WaveForms-SDK-Getting-Started-PY-master')

In [7]:
print(sys.path)

['C:/Users/20245580/LabCode/AD3/WaveForms-SDK-Getting-Started-PY-master/WaveForms-SDK-Getting-Started-PY-master', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\python312.zip', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\DLLs', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\Lib', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1', '', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\Lib\\site-packages', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\Lib\\site-packages\\win32', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\20245580\\AppData\\Local\\anaconda3\\envs\\workspace_1\\Lib\\site-packages\\Pythonwin', 'C:\\Program Files (x86)\\Digilent\\WaveFormsSDK\\samples\\py']


In [8]:
from WF_SDK import device
from WF_SDK import scope
from WF_SDK import wavegen

In [ ]:
from WF_SDK import device, scope, wavegen   # import instruments
 
import matplotlib.pyplot as plt   # needed for plotting
 
"""-----------------------------------------------------------------------"""
 
# connect to the device
device_data = device.open()
 
"""-----------------------------------"""
 
# initialize the scope with default settings
scope.open(device_data)
 
# generate a 10KHz sine signal with 2V amplitude on channel 1
wavegen.generate(device_data, channel=1, function=wavegen.function.sine, offset=0, frequency=10e03, amplitude=2)
 
# record data with the scopeon channel 1
print(scope.record(device_data, channel=1))
 
# reset the scope
scope.close(device_data)
 
# reset the wavegen
wavegen.close(device_data)
 
"""-----------------------------------"""
 
# close the connection
device.close(device_data)

[-2.0003036318673115, -1.999045253875532, -1.999045253875532, -2.0018766043570357, -1.9955847143981382, -1.998416064879642, -1.9996744428714217, -1.9974722813858075, -1.9927533639166342, -1.9943263364063586, -1.993382552912524, -1.9946409309043036, -1.9965284978919728, -1.9946409309043036, -1.993382552912524, -1.9930679584145792, -1.993382552912524, -1.9914949859248547, -1.9914949859248547, -1.9880344464474609, -1.990865796928965, -1.9889782299412955, -1.9892928244392405, -1.9880344464474609, -1.9896074189371853, -1.985203095965957, -1.9886636354433507, -1.985203095965957, -1.984888501468012, -1.9833155289782876, -1.9817425564885631, -1.9833155289782876, -1.982686339982398, -1.9817425564885631, -1.979540395002949, -1.979540395002949, -1.9782820170111695, -1.9785966115091143, -1.9773382335173348, -1.9773382335173348, -1.9729339105461063, -1.9691587765707679, -1.9726193160481615, -1.974192288537886, -1.9704171545625473, -1.9704171545625473, -1.9647544535995394, -1.9650690480974844, -1.96

In [10]:
# reset the scope
scope.close(device_data)
 
# reset the wavegen
wavegen.close(device_data)
 
"""-----------------------------------"""
 
# close the connection
device.close(device_data)